In [1]:
import pandas as pd
import os

Understanding our Data


In [2]:
file_path = r"GCB2022v27_MtCO2_flat.csv"
df = pd.read_csv(file_path)

In [3]:
print("Dataset shape:", df.shape)
print("First 5 rows:\n", df.head(10))

Dataset shape: (63104, 11)
First 5 rows:
        Country ISO 3166-1 alpha-3  Year  Total  Coal  Oil  Gas  Cement  \
0  Afghanistan                AFG  1750    0.0   NaN  NaN  NaN     NaN   
1  Afghanistan                AFG  1751    0.0   NaN  NaN  NaN     NaN   
2  Afghanistan                AFG  1752    0.0   NaN  NaN  NaN     NaN   
3  Afghanistan                AFG  1753    0.0   NaN  NaN  NaN     NaN   
4  Afghanistan                AFG  1754    0.0   NaN  NaN  NaN     NaN   
5  Afghanistan                AFG  1755    0.0   NaN  NaN  NaN     NaN   
6  Afghanistan                AFG  1756    0.0   NaN  NaN  NaN     NaN   
7  Afghanistan                AFG  1757    0.0   NaN  NaN  NaN     NaN   
8  Afghanistan                AFG  1758    0.0   NaN  NaN  NaN     NaN   
9  Afghanistan                AFG  1759    0.0   NaN  NaN  NaN     NaN   

   Flaring  Other  Per Capita  
0      NaN    NaN         NaN  
1      NaN    NaN         NaN  
2      NaN    NaN         NaN  
3      NaN    N

In [4]:
print("Columns:", df.columns.tolist())

Columns: ['Country', 'ISO 3166-1 alpha-3', 'Year', 'Total', 'Coal', 'Oil', 'Gas', 'Cement', 'Flaring', 'Other', 'Per Capita']


In [5]:
print("\nDataset info:\n")
print(df.info())


Dataset info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63104 entries, 0 to 63103
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Country             63104 non-null  object 
 1   ISO 3166-1 alpha-3  61472 non-null  object 
 2   Year                63104 non-null  int64  
 3   Total               62904 non-null  float64
 4   Coal                21744 non-null  float64
 5   Oil                 21717 non-null  float64
 6   Gas                 21618 non-null  float64
 7   Cement              20814 non-null  float64
 8   Flaring             21550 non-null  float64
 9   Other               1620 non-null   float64
 10  Per Capita          18974 non-null  float64
dtypes: float64(8), int64(1), object(2)
memory usage: 5.3+ MB
None


In [6]:
print("\nMissing values per column:\n")
print(df.isnull().sum())


Missing values per column:

Country                   0
ISO 3166-1 alpha-3     1632
Year                      0
Total                   200
Coal                  41360
Oil                   41387
Gas                   41486
Cement                42290
Flaring               41554
Other                 61484
Per Capita            44130
dtype: int64


In [7]:
print("\nYear range:", df['Year'].min(), "-", df['Year'].max())
print("Number of unique countries:", df['Country'].nunique())


Year range: 1750 - 2021
Number of unique countries: 232


In [8]:
print("\nTop 10 countries by record count:")
print(df['Country'].value_counts().head(10))


Top 10 countries by record count:
Country
Afghanistan                       272
Peru                              272
Niger                             272
Nigeria                           272
Niue                              272
Norway                            272
Occupied Palestinian Territory    272
Oman                              272
Pacific Islands (Palau)           272
Pakistan                          272
Name: count, dtype: int64


In [ ]:
emission_columns = ['Total', 'Coal', 'Oil', 'Gas', 'Cement', 'Flaring']
print("\nSummary statistics for emission columns:")
print(df[emission_columns].describe())


Summary statistics for emission columns:
              Total          Coal           Oil           Gas        Cement  \
count  62904.000000  21744.000000  21717.000000  21618.000000  20814.000000   
mean      55.224788     73.968916     55.760624     23.504285      4.330443   
std      824.845435    598.986992    519.034563    247.674772     50.305770   
min        0.000000      0.000000      0.000000      0.000000      0.000000   
25%        0.000000      0.000000      0.091600      0.000000      0.000000   
50%        0.000000      0.271852      1.044240      0.000000      0.022756   
75%        0.549342      6.736411      8.339752      0.581628      0.568502   
max    37123.850352  15051.512770  12345.653374   7921.829472   1672.592372   

            Flaring  
count  21550.000000  
mean       1.712695  
std       16.727067  
min        0.000000  
25%        0.000000  
50%        0.000000  
75%        0.000000  
max      439.253991  


In [ ]:
print("\nUnique special rows in 'Country':")
special_rows = df[df['Country'].str.contains('World|International|Transport', case=False)]
print(special_rows['Country'].unique())


Unique special rows in 'Country':
['International Transport']


In [ ]:
print("\nSample trend for India:")
print(df[df['Country'] == 'India'][['Year', 'Total']].sort_values('Year'))



Sample trend for India:
       Year        Total
25296  1750     0.000000
25297  1751     0.000000
25298  1752     0.000000
25299  1753     0.000000
25300  1754     0.000000
...     ...          ...
25563  2017  2434.867906
25564  2018  2600.446463
25565  2019  2626.459473
25566  2020  2445.011942
25567  2021  2709.683625

[272 rows x 2 columns]


Data Cleaning

In [12]:
df = df[df['Year'].between(2002, 2021)]


In [13]:
df = df[df['Country'] != 'International Transport']


In [ ]:
df = df.drop(columns=['Other', 'Per Capita'])

In [ ]:
df = df.dropna(subset=['Total'])


Mapping ISO Country Codes

In [16]:
iso_df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/2014_world_gdp_with_codes.csv")


In [17]:
iso_df = iso_df[['COUNTRY', 'CODE']].rename(columns={'COUNTRY': 'Country', 'CODE': 'ISO_Code'})

In [18]:
df = df.merge(iso_df, on='Country', how='left')

In [19]:
df['ISO 3166-1 alpha-3'] = df['ISO 3166-1 alpha-3'].fillna(df['ISO_Code'])
df.drop(columns=['ISO_Code'], inplace=True)

Joining our Dataset with Population Data

In [20]:

pop_df = pd.read_csv(r"population.csv")

In [21]:
print(pop_df.head())

  Country Name Country Code  Year      Value
0  Afghanistan          AFG  1960  9035043.0
1  Afghanistan          AFG  1961  9214083.0
2  Afghanistan          AFG  1962  9404406.0
3  Afghanistan          AFG  1963  9604487.0
4  Afghanistan          AFG  1964  9814318.0


In [ ]:
pop_df = pop_df.rename(columns={"Country Name": "Country", "Value": "Population"})

In [ ]:
pop_df = pop_df[pop_df['Year'].between(2002, 2021)]

In [24]:
df= df.merge(pop_df[['Country', 'Year', 'Population']], on=['Country', 'Year'], how='left')


In [25]:
print("After merge, missing population rows:", df['Population'].isnull().sum())


After merge, missing population rows: 1100


In [ ]:
df['Emissions_per_capita'] = df['Total'] / df['Population']


In [27]:
df.to_csv(r"emissions_with_population.csv", index=False)

In [28]:
print(df.head())


       Country ISO 3166-1 alpha-3  Year     Total      Coal       Oil  \
0  Afghanistan                AFG  2002  1.340995  0.055109  0.727438   
1  Afghanistan                AFG  2003  1.559602  0.091813  0.991575   
2  Afghanistan                AFG  2004  1.237247  0.091600  0.908672   
3  Afghanistan                AFG  2005  1.889507  0.106256  1.447280   
4  Afghanistan                AFG  2006  2.159318  0.160944  1.656993   

        Gas    Cement  Flaring  Population  Emissions_per_capita  
0  0.547416  0.011033      0.0  21378117.0          6.272746e-08  
1  0.466408  0.009807      0.0  22733049.0          6.860505e-08  
2  0.227168  0.009807      0.0  23560654.0          5.251327e-08  
3  0.329760  0.006211      0.0  24404567.0          7.742432e-08  
4  0.329204  0.012177      0.0  25424094.0          8.493195e-08  


In [29]:
print(df.columns.tolist())
print(df.dtypes)


['Country', 'ISO 3166-1 alpha-3', 'Year', 'Total', 'Coal', 'Oil', 'Gas', 'Cement', 'Flaring', 'Population', 'Emissions_per_capita']
Country                  object
ISO 3166-1 alpha-3       object
Year                      int64
Total                   float64
Coal                    float64
Oil                     float64
Gas                     float64
Cement                  float64
Flaring                 float64
Population              float64
Emissions_per_capita    float64
dtype: object


In [ ]:
print(df[['Total', 'Population', 'Emissions_per_capita']].describe())

              Total    Population  Emissions_per_capita
count   4620.000000  3.520000e+03          3.520000e+03
mean     283.370938  3.470585e+07          4.948853e-06
std     2297.151090  1.419822e+08          6.576423e-06
min        0.000000  9.623000e+03          0.000000e+00
25%        0.673925  1.346008e+06          6.266451e-07
50%        6.596013  6.816566e+06          2.722178e-06
75%       50.464774  2.050388e+07          6.658775e-06
max    37123.850352  1.414204e+09          6.496179e-05
